In [35]:
def add_round_key(state, sk, size):
    """Add round key operation."""
    return state ^ (sk >> size - 64) & 0xFFFFFFFFFFFFFFFF
state =0x1236_5334_bacf_4232
sk =0xffff_bbbb_aaaa_2123
cy =add_round_key(state,sk,64)
print("this is cypher: "+hex(cy))

this is cypher: 0xedc9e88f10656311


In [36]:
def inv_add_round_key(state, sk, size):
    """Add round key operation."""
    return state ^ (sk >> size - 64) & 0xFFFFFFFFFFFFFFFF
state =0xedc9e88f10656311
sk =0xffff_bbbb_aaaa_2123
pl =inv_add_round_key(state,sk,64)
print("this is plane: "+hex(pl))
    

this is plane: 0x12365334bacf4232


In [37]:
SBOX = [0x7, 0x4, 0xA, 0x9, 0x1, 0xF, 0xB, 0x0,
        0xC, 0x3, 0x2, 0x6, 0x8, 0xE, 0xD, 0x5]


def sbox_nibble(bits, i, N):#(state,0,64)
    """Replaces the i-th nibble (0-base) in N bits with SBOX[nibble]."""
    offset = N - (i + 1) * 4 #60
    nibble = (bits >> offset) & 0xF  # fetch the nibble
    return bits & ~(0xF << offset) | (SBOX[nibble] << offset)  # add back in

def sub_nibbles(state):
    """Apply S-box substitution to all nibbles in the state."""
    for i in range(16):#0-15
        state = sbox_nibble(state, i, 64)#sbox_nibble(state, 0, 64)
    return state
    
state=0xf000_f000_000b_000b
after =(sub_nibbles(state))
print("this is after sub: "+hex(after))

this is after sub: 0x5777577777767776


In [38]:
SBOX = [0x7, 0x4, 0xA, 0x9, 0x1, 0xF, 0xB, 0x0,
        0xC, 0x3, 0x2, 0x6, 0x8, 0xE, 0xD, 0x5]


def inv_sbox_nibble(bits, i, N):#(state,0,64)
    """Replaces the i-th nibble (0-base) in N bits with SBOX[nibble]."""
    offset = N - (i + 1) * 4 #60
    nibble = (bits >> offset) & 0xF  # fetch the nibble
    return bits & ~(0xF << offset) | (SBOX[nibble] << offset)  # add back in

def inv_sub_nibbles(state):
    """Apply S-box substitution to all nibbles in the state."""
    for i in range(16):#0-15
        state = inv_sbox_nibble(state, i, 64)#sbox_nibble(state, 0, 64)
    return state
    
state=0x5777577777767776
after =(inv_sub_nibbles(state))
print("this is after sub: "+hex(after))

this is after sub: 0xf000f000000b000b


In [39]:
def rotate_nibbles(state):
    """Rotate nibbles operation."""
    return (state << 16) & 0xFFFFFFFFFFFFFFFF | (state >> 48)
state=0xaaaa_cccc_bbbb_eeee
after1 = rotate_nibbles(state)
print("this is after rotate_nibbles : "+hex(after1))

this is after rotate_nibbles : 0xccccbbbbeeeeaaaa


In [40]:
def inv_rotate_nibbles(state):
    """Inverse of rotate nibbles operation (right rotation by 16 bits)."""
    return (state >> 16) | ((state & 0xFFFF) << 48)
state=0xcccc_bbbb_eeee_aaaa
after2 = inv_rotate_nibbles(state)
print("this is after inv_rotate_nibbles : "+hex(after2))


this is after inv_rotate_nibbles : 0xaaaaccccbbbbeeee


In [41]:
def mix_columns(bits):
    """Mix columns operation for a 32-bit column."""
    c01 = 0xFF & (bits >> 24)
    c23 = 0xFF & (bits >> 16)
    c45 = 0xFF & (bits >> 8)
    c67 = 0xFF & bits

    def mul2or3(x, n):  # multiplication by 2 or 3 in GF(2^8)
        x = (x << 1) if n == 2 else ((x << 1) ^ x)
        if x > 0xFF:
            return (x ^ 0x1B) & 0xFF
        return x

    s01 = mul2or3(c01, 2) ^ mul2or3(c23, 3) ^ c45 ^ c67
    s23 = c01 ^ mul2or3(c23, 2) ^ mul2or3(c45, 3) ^ c67
    s45 = c01 ^ c23 ^ mul2or3(c45, 2) ^ mul2or3(c67, 3)
    s67 = mul2or3(c01, 3) ^ c23 ^ c45 ^ mul2or3(c67, 2)
    return s01 << 24 | s23 << 16 | s45 << 8 | s67


def mix_nibbles(state):
    """Mix nibbles for the entire state."""
    col1 = mix_columns(state >> 32)
    col2 = mix_columns(state & 0xFFFFFFFF)
    return (col1 << 32) | col2
state=0xaaaa_bbbb_cccc_dddd
result=mix_nibbles(state)
print("this is after mix_nibbles : "+hex(result))

this is after mix_nibbles : 0xaa88bb99cceeddff


In [42]:
def xtime(byte):
    """Shift left by 1 (multiply by x), reduce by 0x11B if overflow."""
    byte <<= 1
    if byte > 0xFF:
        byte ^= 0x11B
    return byte & 0xFF

def mul2or3(x, n):
    """Multiplication by 2 or 3 in GF(2^8)."""
    if n == 2:
        return xtime(x)
    elif n == 3:
        
        return xtime(x) ^ x
    elif n == 9:
        return xtime(xtime(xtime(x))) ^ x
    elif n == 0xB:  # 0xB = 0x9 + 0x2
        return xtime(xtime(xtime(x))) ^ xtime(x) ^ x
    elif n == 0xD:  # 0xD = 0x9 + 0x4
        return xtime(xtime(xtime(x))) ^ xtime(xtime(x)) ^ x
    elif n == 0xE:  # 0xE = 0x8 + 0x4 + 0x2
        return xtime(xtime(xtime(x))) ^ xtime(xtime(x)) ^ xtime(x)
    return x

def inv_mix_columns(bits):
    """Mix columns operation for a 32-bit column."""
    c01 = 0xFF & (bits >> 24)
    c23 = 0xFF & (bits >> 16)
    c45 = 0xFF & (bits >> 8)
    c67 = 0xFF & bits

    # Apply the MixColumns transformation using the multiplication steps.
    s01 = mul2or3(c01, 14) ^ mul2or3(c23, 11) ^ mul2or3(c45, 13) ^ mul2or3(c67, 9)
    s23 = mul2or3(c01, 9) ^ mul2or3(c23, 14) ^ mul2or3(c45, 11) ^ mul2or3(c67, 13)
    s45 = mul2or3(c01, 13) ^ mul2or3(c23, 9) ^ mul2or3(c45, 14) ^ mul2or3(c67, 11)
    s67 = mul2or3(c01, 11) ^ mul2or3(c23, 13) ^ mul2or3(c45, 9) ^ mul2or3(c67, 14)

    return s01 << 24 | s23 << 16 | s45 << 8 | s67

def inv_mix_nibbles(state):
    """Mix nibbles for the entire state (64-bit)."""
    col1 = inv_mix_columns(state >> 32)  # Mix the upper 32 bits
    col2 = inv_mix_columns(state & 0xFFFFFFFF)  # Mix the lower 32 bits
    return (col1 << 32) | col2

# Example usage:
state = 0xaa88bb99cceeddff
result = inv_mix_nibbles(state)
print("This is after mix_nibbles: " + hex(result))





This is after mix_nibbles: 0xaaaabbbbccccdddd


In [43]:
def keySchedule(sk, i, size):
    half_size = size // 2
    a = (sk >> half_size)
    b = sk & int('1' * half_size, 2)

    # Rotate both halves
    a = (a << 8) & int('1' * half_size, 2) | (a >> (half_size - 8))
    b = (b << 8) & int('1' * half_size, 2) | (b >> (half_size - 8))

    # XOR and swap halves
    a ^= b
    a, b = b, a

    # XOR round index shifted into the upper half
    a ^= i << (half_size - 24)

    # Apply S-box substitution to part of b
    for j in range(2, 6):
        b = sbox_nibble(b, j, half_size)

    return a << half_size | b

# Example usage:
key = 0xABCD1234CDEF9876  # Initial key
nr = 12  # Number of rounds
size = 64  # Key size in bits

# Initialize the list with the original key
key_round = [hex(key)]

# Generate all round keys and store in the list
for i in range(1, nr + 1):
    key = keySchedule(key, i, size)
    key_round.append(hex(key))

# Print all keys in the list

print(key_round)


['0xabcd1234cdef9876', '0xef9877cd22c21a66', '0xc21a64225abe26cd', '0xbe26ce5aa41ad598', '0x1ad59ca43c468a1a', '0x468a1f3c934b6d26', '0x4b6d2093c10a42d5', '0xa42d2c167ba1b8a', '0xba1b8267f883166d', '0x831664f898317242', '0x31724898274b621b', '0x4b62102739a2c916', '0xa2c91a39c0e39472']


In [44]:
def encrypt(plaintext, key=[], nr=12, size=64):
    state = plaintext
    # Convert keys to integers if necessary
    sk = [int(k, 16) if isinstance(k, str) else k for k in key]
    
    print("Plaintext: " + hex(state))
    print("Key: " + ", ".join(hex(k) for k in sk))  # Join all round keys as hex strings
    
    for i in range(nr):
        state = add_round_key(state, sk[i], size)
        print(f"After round {i + 1} AddRoundKey: {hex(state)}")
        state = sub_nibbles(state)
        print(f"State after round {i + 1} subNibbles: {hex(state)}")
        state = rotate_nibbles(state)
        print(f"State after round {i + 1} rotateNibbles: {hex(state)}")
        state = mix_nibbles(state)
        print(f"State after round {i + 1} mixNibbles: {hex(state)}")
        print(f"Key for round {i + 1} is {hex(sk[i])}")
    print(f"Key for round 13 is {hex(sk[-1])}")
    state = add_round_key(state, sk[-1], size)
    return state

# Example Usage
plaintext = 0xffff_ffff_ffff_ffff

ciphertext = encrypt(plaintext, key_round)
print("Plaintext:", hex(plaintext))
print("Ciphertext:", hex(ciphertext))


Plaintext: 0xffffffffffffffff
Key: 0xabcd1234cdef9876, 0xef9877cd22c21a66, 0xc21a64225abe26cd, 0xbe26ce5aa41ad598, 0x1ad59ca43c468a1a, 0x468a1f3c934b6d26, 0x4b6d2093c10a42d5, 0xa42d2c167ba1b8a, 0xba1b8267f883166d, 0x831664f898317242, 0x31724898274b621b, 0x4b62102739a2c916, 0xa2c91a39c0e39472
After round 1 AddRoundKey: 0x5432edcb32106789
State after round 1 subNibbles: 0xf19ade869a47b0c3
State after round 1 rotateNibbles: 0xde869a47b0c3f19a
State after round 1 mixNibbles: 0xeb3bbeeb4ebf3fd6
Key for round 1 is 0xabcd1234cdef9876
After round 2 AddRoundKey: 0x4a3c9266c7d25b0
State after round 2 subNibbles: 0x712983abb80eaf67
State after round 2 rotateNibbles: 0x83abb80eaf677129
State after round 2 mixNibbles: 0x4d135191b4db51ae
Key for round 2 is 0xef9877cd22c21a66
After round 3 AddRoundKey: 0x8f0935b3ee657763
State after round 3 subNibbles: 0xc5739f69ddbf00b9
State after round 3 rotateNibbles: 0x9f69ddbf00b9c573
State after round 3 mixNibbles: 0xfc8e8d6b664ebd9a
Key for round 3 is 0xc21a6

In [45]:
def decrypt(ciphertext, key=[], nr=12, size=64):
    state = ciphertext
    # Convert keys to integers if necessary
    sk = [int(k, 16) if isinstance(k, str) else k for k in key]
    
    print("cypher: " + hex(state))
    print("Key: " + ", ".join(hex(k) for k in sk))  # Join all round keys as hex strings
    print(f"Key for round 13 is {hex(sk[-1])}")
    state = inv_add_round_key(state, sk[-1], size)
    for i in range(nr-1, -1, -1):
        print(f"Key for round {i + 1} is {hex(sk[i])}")
        state = inv_mix_nibbles(state)
        print(f"State after round {i + 1} mixNibbles: {hex(state)}")
        state = inv_rotate_nibbles(state)
        print(f"State after round {i + 1} rotateNibbles: {hex(state)}")
        state = inv_sub_nibbles(state)
        print(f"State after round {i + 1} subNibbles: {hex(state)}")
        state = inv_add_round_key(state, sk[i], size)
        print(f"After round {i + 1} AddRoundKey: {hex(state)}")
   
    return state

# Example Usage
ciphertext = 0xcdc0_b51f_1472_2bbe

Plaintext = decrypt(ciphertext , key_round)
print("Plaintext:", hex(Plaintext))
print("Ciphertext:", hex(ciphertext))


cypher: 0xcdc0b51f14722bbe
Key: 0xabcd1234cdef9876, 0xef9877cd22c21a66, 0xc21a64225abe26cd, 0xbe26ce5aa41ad598, 0x1ad59ca43c468a1a, 0x468a1f3c934b6d26, 0x4b6d2093c10a42d5, 0xa42d2c167ba1b8a, 0xba1b8267f883166d, 0x831664f898317242, 0x31724898274b621b, 0x4b62102739a2c916, 0xa2c91a39c0e39472
Key for round 13 is 0xa2c91a39c0e39472
Key for round 12 is 0x4b62102739a2c916
State after round 12 mixNibbles: 0x58345dde743f730e
State after round 12 rotateNibbles: 0x730e58345dde743f
State after round 12 subNibbles: 0x97dfc91feed0195
After round 12 AddRoundKey: 0x421fecb6c74fc883
Key for round 11 is 0x31724898274b621b
State after round 11 mixNibbles: 0x494fe9e871c6e397
State after round 11 rotateNibbles: 0xe397494fe9e871c6
State after round 11 subNibbles: 0xd9301315d3dc048b
After round 11 AddRoundKey: 0xe8425b8df4976690
Key for round 10 is 0x831664f898317242
State after round 10 mixNibbles: 0xecd0a7e71d23be15
State after round 10 rotateNibbles: 0xbe15ecd0a7e71d23
State after round 10 subNibbles: 0x6

In [ ]:
operation = input("Do you want to encrypt or decrypt? (e/d): ").strip().lower()

if operation == 'e':
    plaintext = int(input("Enter the plaintext (in hex, e.g., 0xA3F5B2D68E9C1F72): "), 16)
    initial_key = int(input("Enter the initial key (in hex, e.g., 0x0000000000000000): "), 16)

    nr = 12
    size = 64
    key = initial_key  # Use the integer value of the key
    key_round = [key]   # Start with the initial key as an integer

    # Generate all round keys and store in the list
    for i in range(1, nr + 1):
        key = keySchedule(key, i, size)
        key_round.append(key)  # Store the key as an integer

    ciphertext = encrypt(plaintext, key_round)
    print(f"Ciphertext: {hex(ciphertext)}")

elif operation == 'd':
    ciphertext = int(input("Enter the ciphertext (in hex, e.g., 0x18bcf9d579781cd1): "), 16)
    initial_key = int(input("Enter the initial key (in hex, e.g., 0x0000000000000000): "), 16)

    nr = 12
    size = 64
    key = initial_key  # Use the integer value of the key
    key_round = [key]   # Start with the initial key as an integer

    # Generate all round keys and store in the list
    for i in range(1, nr + 1):
        key = keySchedule(key, i, size)
        key_round.append(key)  # Store the key as an integer

    plaintext = decrypt(ciphertext, key_round)
    print(f"Plaintext: {hex(plaintext)}")

else:
    print("Invalid operation. Please enter 'e' for encrypt or 'd' for decrypt.")


In [31]:
def encrypt_ones(plaintext, key=[], nr=1, size=64):
    state = plaintext
    # Convert keys to integers if necessary
    sk = [int(k, 16) if isinstance(k, str) else k for k in key]
    
    print("Plaintext: " + hex(state))
    print(f"Key for round 1 is {hex(sk[0])}")
    
    
    state = add_round_key(state, sk[0], size)
    print(f"After round 1 AddRoundKey: {hex(state)}")
    state = sub_nibbles(state)
    print(f"State after round  1 subNibbles: {hex(state)}")
    state = rotate_nibbles(state)
    print(f"State after round 1 rotateNibbles: {hex(state)}")
    state = mix_nibbles(state)
    print(f"State after round 1 mixNibbles: {hex(state)}")
    print(f"Key for round 2 is {hex(sk[1])}")
    return state
    
   

# Example Usage
plaintext = 0xa3f5b2d68e9c1f72
key = 0xabcd1234cdef9876  # Initial key
nr=12  # Number of rounds
size = 64  # Key size in bits

# Initialize the list with the original key
key_round = [hex(key)]

# Generate all round keys and store in the list
for i in range(1, nr + 1):
    key = keySchedule(key, i, size)
    key_round.append(hex(key))

ciphertext = encrypt_ones(plaintext, key_round)

print("Ciphertext:", hex(ciphertext))



Plaintext: 0xa3f5b2d68e9c1f72
Key for round 1 is 0xabcd1234cdef9876
After round 1 AddRoundKey: 0x838a0e243738704
State after round  1 subNibbles: 0x7c9c27da1909c071
State after round 1 rotateNibbles: 0x27da1909c0717c9c
State after round 1 mixNibbles: 0x2baad4b8e83af675
Key for round 2 is 0xef9877cd22c21a66
Ciphertext: 0x2baad4b8e83af675


In [34]:
def decrypt_ones(ciphertext, key=[], nr=12, size=64):
    state = ciphertext
    # Convert keys to integers if necessary
    sk = [int(k, 16) if isinstance(k, str) else k for k in key]
    
    print("cypher: " + hex(state))
    
    
    
   
    print(f"Key for round 11 is {hex(sk[1])}")
    print(f"Key for round 12 is {hex(sk[0])}")
    state = inv_mix_nibbles(state)
    print(f"State after round 12 mixNibbles: {hex(state)}")
    state = inv_rotate_nibbles(state)
    print(f"State after round 12 rotateNibbles: {hex(state)}")
    state = inv_sub_nibbles(state)
    print(f"State after round 12 subNibbles: {hex(state)}")
    state = inv_add_round_key(state, sk[0], size)
    print(f"After round 12 AddRoundKey: {hex(state)}")
    
    return state

# Example Usage
ciphertext = 0x2baad4b8e83af675

key = 0xabcd1234cdef9876  # Initial key
nr=12  # Number of rounds
size = 64  # Key size in bits

# Initialize the list with the original key
key_round = [hex(key)]

# Generate all round keys and store in the list
for i in range(1, nr + 1):
    key = keySchedule(key, i, size)
    key_round.append(hex(key))

Plaintext = decrypt_ones(ciphertext , key_round)
print("Plaintext:", hex(Plaintext))




cypher: 0x2baad4b8e83af675
Key for round 11 is 0xef9877cd22c21a66
Key for round 12 is 0xabcd1234cdef9876
State after round 12 mixNibbles: 0x27da1909c0717c9c
State after round 12 rotateNibbles: 0x7c9c27da1909c071
State after round 12 subNibbles: 0x838a0e243738704
After round 12 AddRoundKey: 0xa3f5b2d68e9c1f72
Plaintext: 0xa3f5b2d68e9c1f72
